In [90]:
# Read google drive file
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [91]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import ast

In [92]:
df = pd.read_csv('/content/gdrive/MyDrive/Project - Real Estate/real_estate_data.csv')

In [93]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
np.random.seed(3)

In [94]:
df.isna().sum()

PropertyName          0
PropertySubName       0
NearbyLocations       0
LocationAdvantages    0
Link                  0
PriceDetails          0
TopFacilities         0
dtype: int64

In [95]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PropertyName        247 non-null    object
 1   PropertySubName     247 non-null    object
 2   NearbyLocations     247 non-null    object
 3   LocationAdvantages  247 non-null    object
 4   Link                247 non-null    object
 5   PriceDetails        247 non-null    object
 6   TopFacilities       247 non-null    object
dtypes: object(7)
memory usage: 13.6+ KB


In [97]:
df['PropertyName'].nunique()

247

In [98]:
# the recommender system will be sum of 3 features : Location Advantage, Price details, Top facilities

In [99]:
df.loc[22] #dropping an null row

PropertyName                PropertyName
PropertySubName          PropertySubName
NearbyLocations          NearbyLocations
LocationAdvantages    LocationAdvantages
Link                                Link
PriceDetails                PriceDetails
TopFacilities              TopFacilities
Name: 22, dtype: object

In [100]:
df = df.drop(index=22)

In [101]:
df.columns

Index(['PropertyName', 'PropertySubName', 'NearbyLocations', 'LocationAdvantages', 'Link', 'PriceDetails', 'TopFacilities'], dtype='object')

In [102]:
df.isna().sum()

PropertyName          0
PropertySubName       0
NearbyLocations       0
LocationAdvantages    0
Link                  0
PriceDetails          0
TopFacilities         0
dtype: int64

In [103]:
df.duplicated().sum()

0

In [104]:
df.iloc[2].NearbyLocations

"['AIPL Business Club Sector 62', 'Heritage Xperiential Learning School', 'CK Birla Hospital', 'Paras Trinity Mall Sector 63', 'Rapid Metro Station Sector 56']"

In [105]:
df.iloc[2].LocationAdvantages

"{'AIPL Business Club Sector 62': '2.7 Km', 'Heritage Xperiential Learning School': '2 Km', 'CK Birla Hospital': '2.5 Km', 'Paras Trinity Mall Sector 63': '3.5 Km', 'Rapid Metro Station Sector 56': '3.8 Km', 'De Adventure Park': '6.8 Km', 'Golf Course Ext Rd': '99 Meter', 'DoubleTree by Hilton Hotel Gurgaon': '3.6 Km', 'KIIT College of Engineering Sohna Road': '8.4 Km', 'Mehrauli-Gurgaon Road': '11.8 Km', 'Indira Gandhi International Airport': '21.1 Km', 'Nirvana Rd': '160 Meter', 'TERI Golf Course': '8.7 Km'}"

In [106]:
df.iloc[1].PriceDetails

"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}"

In [107]:
df.iloc[2].TopFacilities

"['Terrace Garden', 'Gazebo', 'Fountain', 'Amphitheatre', 'Party Lawn', 'Basketball Court', 'Badminton Court', 'Yoga/Meditation Area', 'Indoor Games']"

In [108]:
df[['PropertyName','TopFacilities']]['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [109]:
def extract_list(s):
    return re.findall(r"'(.*?)'", s)

df['TopFacilities'] = df['TopFacilities'].apply(extract_list)

In [110]:
df.head(2)

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","[Swimming Pool, Salon, Restaurant, Spa, Cafete..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","[Bowling Alley, Mini Theatre, Manicured Garden..."


In [111]:
df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join)

In [112]:
df['FacilitiesStr'][0]

'Swimming Pool Salon Restaurant Spa Cafeteria Sun Deck 24x7 Security Club House Gated Community'

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

In [114]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['FacilitiesStr'])

In [115]:
tfidf_matrix.toarray()[0]

array([0.        , 0.        , 0.        , 0.18809342, 0.18809342,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [116]:
cosine_TopFacilities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [117]:
cosine_TopFacilities.shape

(246, 246)

In [118]:
# the function fetches the similar property in terms of top facilities

def recommend_properties(property_name, cosine_sim=cosine_TopFacilities):
    # Get the index of the property that matches the name
    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # Get the pairwise similarity scores with that property
    sim_scores = list(enumerate(cosine_TopFacilities[idx]))

    # Sort the properties based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar properties
    sim_scores = sim_scores[1:6]

    # Get the property indices
    property_indices = [i[0] for i in sim_scores]

    recommendations_df = pd.DataFrame({
        'PropertyName': df['PropertyName'].iloc[property_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar properties
    return recommendations_df

In [119]:
recommend_properties("DLF The Arbour")

,PropertyName,SimilarityScore
64,Ace Palm Floors,"(63, 0.45293820624419556)"
217,Yashika 104,"(216, 0.41996063229267827)"
93,JMS The Nation,"(92, 0.4166584649363288)"
154,India Rashtra,"(153, 0.39895423468019414)"
0,Smartworld One DXP,"(0, 0.388850461994329)"


In [120]:
df[['PropertyName','PriceDetails']]['PriceDetails'][1]

"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}"

In [121]:
import pandas as pd
import json

# Load the dataset
df_appartments = pd.read_csv('/content/gdrive/MyDrive/Project - Real Estate/real_estate_data.csv').drop(22)

# Function to parse and extract the required features from the PriceDetails column
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted
# Apply the refined parsing and generate the new DataFrame structure
data_refined = []

for _, row in df_appartments.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])

    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}

    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')

    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')

In [122]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [123]:
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,building type_3 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,building type_4 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,building type_5 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,building type_6 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.00,1370.00,2.0000,2.4000,Apartment,1850.00,2050.00,2.2500,3.59,Apartment,2600.00,2600.00,3.24,4.56,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,1605.00,2170.00,2.2000,3.03,Apartment,2248.00,2670.00,3.08,3.73,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Independent Floor,1800.00,3150.00,2.4300,15.75,Independent Floor,2750.00,4500.00,3.36,22.50,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Land,500.00,4329.00,2.0500,41.1300
Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.00,1692.00,1.5500,3.2100,Apartment,1711.00,2343.00,1.7600,4.79,Apartment,2423.00,2963.00,2.50,6.06,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.00,1118.00,0.9301,1.0600,Independent Floor,1235.00,1530.00,1.1200,1.45,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Whiteland The Aspen,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,2290.00,2937.00,3.0700,3.96,Apartment,3243.00,12613.00,4.35,17.39,Apartment,6096.00,7582.00,8.21,10.24,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Bestech Altura,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,2015.00,2150.00,1.5300,1.85,Apartment,2250.00,2675.00,1.71,2.46,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Elan The Presidential,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,1347.00,2600.00,4.0500,8.88,Apartment,1692.00,3100.00,4.89,10.59,Apartment,2275.00,4100.00,6.00,14.00,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 92,None,NaN,NaN,NaN,NaN,Independent Floor,959.00,1145.00,1.2600,1.5100,Independent Floor,1091.00,1535.00,0.8646,4.01,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [124]:
df_final_refined_v2.columns

Index(['building type_1 BHK', 'area low 1 BHK', 'area high 1 BHK', 'price low 1 BHK', 'price high 1 BHK', 'building type_2 BHK', 'area low 2 BHK', 'area high 2 BHK', 'price low 2 BHK', 'price high 2 BHK', 'building type_3 BHK', 'area low 3 BHK', 'area high 3 BHK', 'price low 3 BHK', 'price high 3 BHK', 'building type_4 BHK', 'area low 4 BHK', 'area high 4 BHK', 'price low 4 BHK', 'price high 4 BHK', 'building type_5 BHK', 'area low 5 BHK', 'area high 5 BHK', 'price low 5 BHK', 'price high 5 BHK', 'building type_6 BHK', 'area low 6 BHK', 'area high 6 BHK', 'price low 6 BHK', 'price high 6 BHK', 'building type_1 RK', 'area low 1 RK', 'area high 1 RK', 'price low 1 RK', 'price high 1 RK', 'building type_Land', 'area low Land', 'area high Land', 'price low Land', 'price high Land'], dtype='object')

In [125]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()

In [126]:
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [127]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns=categorical_columns, drop_first=True)

In [128]:
ohe_df.fillna(0,inplace=True)

In [129]:
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,area low Land,area high Land,price low Land,price high Land,building type_1 BHK_Service Apartment,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.00,0.00,0.00,0.0000,1370.00,1370.00,2.0000,2.4000,1850.00,2050.00,2.2500,3.59,2600.00,2600.00,3.24,4.56,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
M3M Crown,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,1605.00,2170.00,2.2000,3.03,2248.00,2670.00,3.08,3.73,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
Adani Brahma Samsara Vilasa,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,1800.00,3150.00,2.4300,15.75,2750.00,4500.00,3.36,22.50,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,500.00,4329.00,2.0500,41.1300,0,0,0,1,0,0,1,0,0,0,0
Sobha City,0.00,0.00,0.00,0.0000,1381.00,1692.00,1.5500,3.2100,1711.00,2343.00,1.7600,4.79,2423.00,2963.00,2.50,6.06,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
Signature Global City 93,0.00,0.00,0.00,0.0000,981.00,1118.00,0.9301,1.0600,1235.00,1530.00,1.1200,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,1,0,1,0,0,0,0,0,0,0
Whiteland The Aspen,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,2290.00,2937.00,3.0700,3.96,3243.00,12613.00,4.35,17.39,6096.00,7582.00,8.21,10.24,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
Bestech Altura,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,2015.00,2150.00,1.5300,1.85,2250.00,2675.00,1.71,2.46,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
Elan The Presidential,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,1347.00,2600.00,4.0500,8.88,1692.00,3100.00,4.89,10.59,2275.00,4100.00,6.00,14.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0
Signature Global City 92,0.00,0.00,0.00,0.0000,959.00,1145.00,1.2600,1.5100,1091.00,1535.00,0.8646,4.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,1,0,1,0,0,0,0,0,0,0


In [130]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns=ohe_df.columns, index=ohe_df.index)

In [131]:
ohe_df_normalized.head()

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK,area low 6 BHK,area high 6 BHK,price low 6 BHK,price high 6 BHK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,area low Land,area high Land,price low Land,price high Land,building type_1 BHK_Service Apartment,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,0.807098,0.515061,0.602838,0.212073,0.383381,0.242019,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,0.772095,0.342903,0.382746,0.243172,0.341443,0.108677,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,0.933110,4.253341,0.696627,1.056178,0.414835,3.124152,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,0.369534,1.828502,0.643875,7.635886,-0.111111,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,0.464065,0.883970,0.492166,0.373342,0.189416,0.483000,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,0.016022,-0.142827,-1.022842,-0.943016,-0.465873,-0.490563,-0.468954,-0.460463,-0.248049,-0.235915,-0.125582,-0.118934,-0.077649,-0.073387,-0.105157,-0.10253,-0.090521,-0.082725,-0.447044,-0.371421,-0.195703,-0.240489,-0.111111,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [132]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_PriceDetails = cosine_similarity(ohe_df_normalized)

In [133]:
cosine_PriceDetails.shape

(246, 246)

In [134]:
def recommend_properties_with_scores(property_name, top_n=247):

    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_PriceDetails[ohe_df_normalized.index.get_loc(property_name)]))

    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]

    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()

    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })

    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Golf Hills')

,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128
5,Sobha City,0.928748
6,Unitech Harmony,0.925164
7,Corona Optus,0.919231
8,Puri Emerald Bay,0.917345
9,Ireo Skyon,0.915991


In [135]:
df[['PropertyName','LocationAdvantages']]['LocationAdvantages'][0]

"{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}"

In [136]:
def extract_dict(s):
    # Using a regular expression to extract key-value pairs enclosed in curly braces
    matches = re.findall(r"{.*?}", s)

    # Converting each match to a dictionary using eval (or you can use json.loads)
    dicts = [eval(match) for match in matches]

    # Combining dictionaries into a single dictionary
    result_dict = {}
    for d in dicts:
        result_dict.update(d)

    return result_dict

# Apply the function to the 'TopFacilities' column
df['LocationAdvantages'] = df['LocationAdvantages'].apply(extract_dict)

In [137]:
df['LocationAdvantages'][6]

{'Mount Olympus Junior School': '1 Km',
 'Miracles Apollo Hospital': '4.8 Km',
 'Sapphire 83 Mall': '5.4 Km',
 'NH -8': '6.5 Km',
 'Savoy Suites, Manesar': '6.9 Km',
 'Golden Greens Golf & Resorts': '8.2 Km',
 'IMT Manesar': '9.2 Km',
 'Amity University Gurugram': '12 Km',
 'Golf Course Extension Road': '13.6 Km',
 'Indira Gandhi International Airport': '31 Km'}

In [138]:
# As there are 1K+ values, the value mapping will be provided to values with frequency more than 1

# property_names = pd.Series(property_names)
# property_names.value_counts()

In [139]:
# common_beginning = 'IMT'

# # Check if values have a similar beginning
# similar_beginning_mask = property_names.str.contains(common_beginning)

# # Print the original Series and the unique values with a similar beginning

# print("\nValues with a Similar Beginning:")
# print(property_names[similar_beginning_mask].unique())

In [140]:
df['LocationAdvantages'][9]

{'Imperia Mindspace': '140 Meter',
 'AIPL Business Tower': '160 Meter',
 'Heritage School': '1.3 KM',
 'Lotus Valley Intl School, Gurgaon': '2.6 KM',
 'Gurugram University': '3.6 KM',
 'Sector 55-56 Metro Station': '3.9 KM',
 'Park Hospital': '5.5 KM',
 'Omaxe Gurgaon Mall': '5.8 KM',
 'Sushant University': '6.1 KM',
 'Badshahpur Sohna Rd Hwy,Sector 48': '7.5 KM',
 'Indira Gandhi Intl Airport': '24.8 KM'}

In [141]:
def distance_to_meters(distance_str):
    try:
        if 'Km' in distance_str or 'KM' in distance_str:
            return float(distance_str.split()[0]) * 1000
        elif 'Meter' in distance_str or 'meter' in distance_str:
            return float(distance_str.split()[0])
        else:
            return None
    except:
        return None

In [142]:
# Assuming df['LocationAdvantages'] contains string representations of dictionaries
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in (row['LocationAdvantages']).items():
        distances[location] = distance_to_meters(distance)
    location_matrix[index] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,"Park Hospital, Palam Vihar",Palam Vihar Halt Railway Station,Dwarka Sector 21 Metro Station,Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,Rapid Metro Station Sector 56,De Adventure Park,Golf Course Ext Rd,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,IGI Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,Holiday Inn Gurugram Sector 90,Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,"Dwarka Expy, Sector 109","Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi Intl Airport,Royal Institute Of Science & Management,Pataudi Road,Holiday Inn Sector 90,RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,Sector 55-56 Metro Station,Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi Int. Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,Sector 55/56 Metro Station,Mavens Inn,Sanar International Hospital,Sector 53-54 Metro Station,"IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Ext Road,"Heritage Xperiential Learning, CRPF Rd",Sector 54 Chowk Metro Station

In [143]:
# Define the dictionary to map misspelled names to correct names

airport_missname = {'Indira Gandhi Intl Airport': 'Indira Gandhi International Airport','Indira Gandhi Int. Airport': 'Indira Gandhi International Airport', 'Indira Gandhi Airport': 'Indira Gandhi International Airport', 'IGI Airport' : 'Indira Gandhi International Airport', 'Airport' : 'Indira Gandhi International Airport' }

dwarka_missname = {'Dwarka Expy' : 'Dwarka Expressway', 'Dwarka Expressway' : 'Dwarka Expressway', 'Dwarka Expy, Sector 109' : 'Dwarka Expressway', 'Dwarka Expy, Sector 88' : 'Dwarka Expressway', 'Dwarka Expy, Dhanwapur Village' : 'Dwarka Expressway','Dwarka Expy/Northern Peripheral Rd' : 'Dwarka Expressway', 'Dwarka Expressway Link Road' : 'Dwarka Expressway', 'Dwarka Expy, Block D, New Palam Vihar' : 'Dwarka Expressway'}

sky_missname = {'SkyJumper Trampoline Park Gurgaon': 'SkyJumper Trampoline Park'}

park_missname = {'Park Hospital, Palam Vihar':'Park Hospital', 'Park Inn, Gurgaon':'Park Inn', 'Park inn': 'Park Inn' }

garhi_missname = {'Garhi harsaru railway station Gurgaon' : 'Garhi Harsaru Junction'}

gurugram_uri_missname = {'Gurugram University Sector 87' : 'Gurugram University', 'Gurugram University Kankrola' : 'Gurugram University' }

metro_missname = {'Dwarka Sector 21 Metro Station' : 'Dwarka Sector 21, Metro Station', 'Rapid Metro Station Sector 56' : 'Sector 55-56, Metro Station', 'Sector 55-56 Metro Station': 'Sector 55-56, Metro Station', 'Sector 55/56 Metro Station' : 'Sector 55-56, Metro Station', 'Sector 53-54 Metro Station' : 'Sector 53-54, Metro Station', 'Sector 55 Metro Station' : 'Sector 55-56, Metro Station', 'Sector 53 Metro Station' : 'Sector 53-54, Metro Station', 'Sector 42-43 Metro Station' : 'Sector 42-43, Metro Station', 'Sector 55-56 Rapid Metro Station':'Sector 55-56, Metro Station', 'Huda Metro Station (Gurugram)' :'Huda Metro Station', 'Sector 42-43 Rapid Metro Station' :  'Sector 42-43, Metro Station', 'Sector 53/54 Metro Station' : 'Sector 55-56, Metro Station', 'HUDA Metro Station' :'Huda Metro Station', 'Rapid Metro Station Sector 55 56' :  'Sector 55-56, Metro Station'  }

gold_course_missname = {'Golf Course Ext Rd': 'Golf Course Extension Road', 'Golf Course Ext Road' : 'Golf Course Extension Road', 'Golf Course Road' : 'Golf Course Extension Road', 'Golf Course Extension Rd' : 'Golf Course Extension Road' , 'Golf Course Extension' : 'Golf Course Extension Road', 'Golf Corse Ext. Rd.' : 'Golf Course Extension Road', 'Golf course extension road' : 'Golf Course Extension Road', 'Golf Course Rd' : 'Golf Course Extension Road', 'Golf Course Extn Road' : 'Golf Course Extension Road' }

airia_missname = {'Airia Mall Sector 68' : 'Airia Mall' }

dpg_missname = {'DPG Degree College' : 'DPG Degree and Engineering College', 'DPGITM Engineering College Sector 34': 'DPG Degree and Engineering College', 'DPG Institute of Technology' : 'DPG Degree and Engineering College'}

sapphire_missname = {'Sapphire 83 Mall Sector 83': 'Sapphire 83 Mall', 'Sapphire 83' : 'Sapphire 83 Mall'}

tau_devlal_missname = {'Tau Devi Lal Sports Complex': 'Tau DeviLal Sports Complex'}

imt_missname = {'NH-8, IMT Manesar': 'IMT Manesar',  'NH-8 IMT Manesar' : 'IMT Manesar',  'IMT Office Sohna': 'IMT Sohna' }

holiday_inn_missname = {'Holiday Inn': 'Holiday Inn Gurugram',  'Holiday Inn Express Gurugram' : 'Holiday Inn Gurugram',  'Holiday Inn Express Gurugram Sec 50': 'Holiday Inn Express, Sector 50', 'Holiday Inn Express Gurugram Sector 50' : 'Holiday Inn Express, Sector 50', 'Holiday Inn Gurugram Sector 90' : 'Holiday Inn, Sector 90', 'Holiday Inn Hotel Sector 90' : 'Holiday Inn, Sector 90', 'Holiday Inn Sector 90': 'Holiday Inn, Sector 90'}

In [144]:
location_df.rename(columns=airport_missname, inplace=True)
location_df.rename(columns=dwarka_missname, inplace=True)
location_df.rename(columns=sky_missname, inplace=True)
location_df.rename(columns=park_missname, inplace=True)
location_df.rename(columns=garhi_missname, inplace=True)
location_df.rename(columns=gurugram_uri_missname, inplace=True)
location_df.rename(columns=metro_missname, inplace=True)
location_df.rename(columns=gold_course_missname, inplace=True)
location_df.rename(columns=airia_missname, inplace=True)
location_df.rename(columns=dpg_missname, inplace=True)
location_df.rename(columns=sapphire_missname, inplace=True)
location_df.rename(columns=tau_devlal_missname, inplace=True)
location_df.rename(columns=imt_missname, inplace=True)
location_df.rename(columns=holiday_inn_missname, inplace=True)

In [145]:
# Display the first few rows
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expressway,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,Park Hospital,Palam Vihar Halt Railway Station,"Dwarka Sector 21, Metro Station",Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,"Sector 55-56, Metro Station",De Adventure Park,Golf Course Extension Road,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,Indira Gandhi International Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,"Holiday Inn, Sector 90",Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,Dwarka Expressway,"Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi International Airport,Royal Institute Of Science & Management,Pataudi Road,"Holiday Inn, Sector 90",RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,"Sector 55-56, Metro Station",Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi International Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,"Sector 55-56, Metro Station",Mavens Inn,Sanar International Hospital,"Sector 53-54, Metro Station","IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree and Engineering College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Extension Road,"Herit

In [146]:
location_df.fillna(0,inplace=True)

In [147]:
# Display the first few rows
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expressway,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,Fun N Food Waterpark,Accenture DDC5,DPSG Palam Vihar Gurugram,Park Hospital,Palam Vihar Halt Railway Station,"Dwarka Sector 21, Metro Station",Dwarka Expressway,Fun N Food Water Park,Tau DeviLal Sports Complex,Hyatt Place,Altrade Business Centre,AIPL Business Club Sector 62,Heritage Xperiential Learning School,CK Birla Hospital,Paras Trinity Mall Sector 63,"Sector 55-56, Metro Station",De Adventure Park,Golf Course Extension Road,DoubleTree by Hilton Hotel Gurgaon,KIIT College of Engineering Sohna Road,Mehrauli-Gurgaon Road,Nirvana Rd,TERI Golf Course,The Shikshiyan School,WTC Plaza,Luxus Haritma Resort,BSF Golf Course,Rions Hospital,Gurgaon,Dwarka Sector 21,Nehru Stadium,Fun N Food WaterPark,Indira Gandhi International Airport,Vasant Kunj,Pranavananda Int. School,DLF Site central office,"Holiday Inn, Sector 90",Krishna Hospital,Royal Institute Of Science,Sapphire 83 Mall,NH48,Garhi Harsaru Junction,Manesar Golf Course,AapnoGhar,Vega Schools NH-8,DLF Corporate Greens,Miracles Apollo Cradle Hospital,Hyatt Regency Gurugram,NH 48,Golden Greens Golf & Resorts Limited,Mount Olympus Junior School,Miracles Apollo Hospital,NH -8,"Savoy Suites, Manesar",Golden Greens Golf & Resorts,IMT Manesar,Amity University Gurugram,Golf Course Extension Road,Dwarka Expressway,"Euro International School, Sector- 109",Jai Sai Ram Hospital,Aryan Hospital,Idea Cosmic Plaza,Indira Gandhi International Airport,Royal Institute Of Science & Management,Pataudi Road,"Holiday Inn, Sector 90",RPS International School,Aarvy Healthcare Hospital,Iris Broadway Mall,Imperia Mindspace,AIPL Business Tower,Heritage School,"Lotus Valley Intl School, Gurgaon",Gurugram University,"Sector 55-56, Metro Station",Omaxe Gurgaon Mall,Sushant University,"Badshahpur Sohna Rd Hwy,Sector 48",Naurangpur Cricket Stadium,Naurangpur Road,National Highway 48,Vatika Town Square-INXT,Ompee International School,Manesar Bus Stand,Yashlok Medical Centre,Euro International School,WorldMark Gurgaon,Capital Cyberscape,The Shriram Millennium School,DoubleTree by Hilton Hotel,Badshahpur Sohna Hwy,Nakhrola Stadium,Delhi - Jaipur Expressway,Vatika Town Square-INXT Mall,Savoy Suites,Bal Bharati Public School,Vatika Business Centre,Indira Gandhi International Airport,St. Xavier's High School,Miracles Apollo Cradle,Ambience Mall New,NH8,Hyatt Regency Gurgaon,Delhi Public School,Elan Miracle Mall,Miracles Apollo Cradle Spectra Hospital,Agri Business Management Collage,Delhi Jaipur Expressway,Grand Hyatt Gurgaon,Duke Horse Riding Club,PVR Drive In Cinema,W Pratiksha Hospital,Metro World Mall,Unicosmos School,Faridabad Gurgaon Road,Sohna Road,Bestech Business Tower,Appu Ghar,SkyJumper Trampoline Park,Axis Bank,KMP Expressway,Karma Lakelands,Jungle Safari & Trails,DPS Manesar,Medanta Hospital,Faridabad - Gurgaon Road,Lingaya's Lalita Devi Institute,ASF Insignia SEZ,Banjara Market Gurugram,Central Plaza Mall,"Paras Hospitals, Gurgaon",Badshahpur Sohna Rd Hwy,Vega School,Indian School of Hospitality,Vatika City Centre,Aatish Hospital,Info Technology Park Phase 2,Huda Metro Station,Southern Peripheral Rd,Global Ways School,Radisson Hotel,NH 248A,"Sector 55-56, Metro Station",Mavens Inn,Sanar International Hospital,"Sector 53-54, Metro Station","IILM University, Gurugram",The Banyan Tree World School,The Big Tree Cafe,DLF Golf and Country Club,"Delhi Public School, Sector 84",Aarvy Hospital,DPG Degree and Engineering College,Shivani public school,Baghera University,Kutumbh Hospital,Bijwasan Railway Station,Global Foyer Mall,Phase 2 Metro Station,Gurgaon Dreamz Mall,"Metro Hospital, Palam Vihar",Delhi Ajmer Expressway,Infinity Business Park,Huda metro station,Rion's Hospital,"Euro International School, Sector- 109.",Golf Course Extension Road,"Herit

In [148]:
max_value = location_df.max().max()
print("Maximum value in the DataFrame:", max_value)

Maximum value in the DataFrame: 54500.0


In [149]:
location_df = location_df.groupby(by=location_df.columns, axis=1).sum()

In [150]:
location_df.head()

,AIIMS,AIIMS Jhajjar,AIPL Business Centre,AIPL Business Club,AIPL Business Club Sector 62,AIPL Business Co Working Space,AIPL Business Tower,AIPL Joy Street Mall,APJ Abdul Kalam Park,ASF Insignia SEZ,Aapno Ghar,AapnoGhar,Aarvy Healthcare,Aarvy Healthcare Hospital,Aarvy Healthcare Super Speciality,Aarvy Hospital,Aatish Hospital,Accenture DDC5,Adarsh Senior Secondary School,"Adarsh public school,Garhi Harsaru",Agri Business Management Collage,Airia Mall,Ajit Stadium Dhanwapur,Alfaa Health Care Hospital,Alpine Convent School,Alpine Hospital,Alpine School,Altrade Business Centre,Aman Hospital,Aman Hospital & Surgical Centre,Ambience Mall,Ambience Mall New,Ambience Public School,American Express,Amity,Amity University,Amity University Gurugram,Amma Hospital,Anand Multispeciality Hospital,Anand Preschool,Ananta Hospital,Ansal Plaza,Anya Gurgaon,Ap Sports cricket ground,Apex Plus Hospital,Apollo,Apollo Pharmacy,Approved Sector 37 Mero Station,Appu Ghar,Appu Ghar Water Park,Aradhya Cricket Club Gurgaon,Aravali Adventure Hill,Aravalli Hill View Point,Aravalli Hills,Arc Hospital,Arc Multi Speciality,Arc Multi Speciality Hospital,Ardee Mall,Artemis,Artemis Hospital,Artemis Hospital Gurgaon,Artimis hospital,Arvy Hospital,Aryan Hospital,Ascendas OneHub Gurgaon,Ascendas OneHub Gurgaon Business Park,Ashiana Anmol Kid Centric Homes,Ashoka International School,Athena,Au Grand Air,Axis Bank,Axis Bank ATM,"Axis Bank, Sohna Rd",BM College of Technology,BM College of Technology & Mgmt,BML Munjal University (BMU),BOB ATM,BSF Golf Course,Baba Kanala Chowk,Badsa AMS Hospital,Badshahpur Sohna Hwy,Badshahpur Sohna Rd Hwy,"Badshahpur Sohna Rd Hwy, Haryana","Badshahpur Sohna Rd Hwy, Malibu Town","Badshahpur Sohna Rd Hwy, Raghav Vatika","Badshahpur Sohna Rd Hwy, Rajoria Ngr","Badshahpur Sohna Rd Hwy, Sector 68","Badshahpur Sohna Rd Hwy,Sector 48",Badshapur Sohna Highway,Baghera University,Bajghera Road,Bal Bharati Public School,Balaji Hospital,Bamroli Cricket Ground,Banjara Market Gurugram,Bank Of Baroda,Bank Of Baroda ATM,Basai Dhancourt Railway Station,Basai Dhankot,Basai Dhankot Railway Station,Basai Enclave Park,Basai Metro Station,Basai Road,Basant Valley Global School,Best IVF Centre,Bestech Business Tower,Bestech Central Square Mall,Bharat Petrol Pump,Bharat Petroleum Petrol Pump,Bharat Petroleum Retail Outlet,Bharat Petroleum Shree Shyam Filling,Bharat Ram Global School,Bharat Singh fuel company,Bharti International Convent School,Bhondsi Nature Park,BigBazaar,Bijwasan Railway Station,Biryani Shah,Blue Bells Public School,Broadways International School,CANARA BANK,CBR Cricket Ground,CD International School,CK Birla Hospital,"CK Birla Hospital, Gurgaon",CNG Petrol Pump,Cambridge College Of Education,Cambridge Montessori,Cambridge Montessori Preschool,Cambridge Pre-School,Canara Bank,Canara Bank - Nawada Fatehpur,Canara Bank ATM,Canara Bank New Palam Vihar,"Candor TechSpace, Sector 48",Candor Techspace,Capital Business Park,Capital Cyberscape,Captain Chandan Lal Marg,Central Bank Of India Sohna Rd,Central Park Flower Valley,Central Park II Road,Central Park Resorts,"Central Park, Sohna Rd",Central Peripheral Road,Central Plaza Mall,Centrum Plaza,Chauma Road,Cherub's Cradle,Children Park,Chirag Hospital,Choice pharmacy,Citibank ATM,City Hospital,City Square,Civil Hospital,Cloudnine Hospital Sector 47,CoNexus.Life B35,Colonel's Central Academy,Conscient One,Conscient One Mall,Cool Deck Coffee,Country Inn,Country Inn & Suites By Radisson,Country Inn & Suites by Radisson,Country Inn and Suites by Radisson,Creative Tennis Academy,Cricket Academy,Cyber ​​Park,DLC Cricket Ground,DLF Corporate Greens,DLF Corporate Park,DLF Cyber City,DLF Golf and Country Club,DLF Grand Mall,DLF Linear Park,DLF Site central office,DLF World Tech Park,DLF5 Summit Plaza,DPG Degree and Engineering College,DPS,DPS International Edge,DPS International School,DPS Manesar,DPS Sector 103,DPSG Palam Vihar,DPSG Palam Vihar Gurugram,DSD College,Damdama Lake,Damdama Lake Rd,Damdama More

In [151]:
max_value = location_df.max().max()
print("Maximum value in the DataFrame:", max_value)

Maximum value in the DataFrame: 54500.0


In [152]:
location_df['Indira Gandhi International Airport']

0      14700.0
25     15600.0
37     20800.0
69     16100.0
9      24800.0
11     22700.0
19     25200.0
39     21000.0
43     24200.0
46         0.0
47     23600.0
48     16400.0
65         0.0
68     24100.0
72     22700.0
73     25600.0
91     25300.0
100        0.0
101    26500.0
110        0.0
127    22800.0
135    25400.0
145    18900.0
154    28500.0
158        0.0
166    22500.0
169    19000.0
177    21500.0
221    25800.0
222        0.0
232    22800.0
233        0.0
234    22500.0
235    22800.0
240    24100.0
7      17800.0
14     27700.0
17     21300.0
24         0.0
26     20400.0
32     19500.0
40     24100.0
52     19300.0
55     21800.0
56     17500.0
62         0.0
71     18100.0
74     23700.0
87     18900.0
103    11000.0
131    22800.0
134    22800.0
148    35300.0
168        0.0
170    23200.0
191        0.0
215    25000.0
244    21700.0
245        0.0
1      14100.0
3      19300.0
44     22100.0
58     14700.0
88     19600.0
102    17700.0
119    15500.0
183    150

In [153]:
# no duplicate columns
df.columns[df.columns.duplicated()].unique()

Index([], dtype='object')

In [154]:
location_df.index = df.PropertyName

In [155]:
location_df.head()

,AIIMS,AIIMS Jhajjar,AIPL Business Centre,AIPL Business Club,AIPL Business Club Sector 62,AIPL Business Co Working Space,AIPL Business Tower,AIPL Joy Street Mall,APJ Abdul Kalam Park,ASF Insignia SEZ,Aapno Ghar,AapnoGhar,Aarvy Healthcare,Aarvy Healthcare Hospital,Aarvy Healthcare Super Speciality,Aarvy Hospital,Aatish Hospital,Accenture DDC5,Adarsh Senior Secondary School,"Adarsh public school,Garhi Harsaru",Agri Business Management Collage,Airia Mall,Ajit Stadium Dhanwapur,Alfaa Health Care Hospital,Alpine Convent School,Alpine Hospital,Alpine School,Altrade Business Centre,Aman Hospital,Aman Hospital & Surgical Centre,Ambience Mall,Ambience Mall New,Ambience Public School,American Express,Amity,Amity University,Amity University Gurugram,Amma Hospital,Anand Multispeciality Hospital,Anand Preschool,Ananta Hospital,Ansal Plaza,Anya Gurgaon,Ap Sports cricket ground,Apex Plus Hospital,Apollo,Apollo Pharmacy,Approved Sector 37 Mero Station,Appu Ghar,Appu Ghar Water Park,Aradhya Cricket Club Gurgaon,Aravali Adventure Hill,Aravalli Hill View Point,Aravalli Hills,Arc Hospital,Arc Multi Speciality,Arc Multi Speciality Hospital,Ardee Mall,Artemis,Artemis Hospital,Artemis Hospital Gurgaon,Artimis hospital,Arvy Hospital,Aryan Hospital,Ascendas OneHub Gurgaon,Ascendas OneHub Gurgaon Business Park,Ashiana Anmol Kid Centric Homes,Ashoka International School,Athena,Au Grand Air,Axis Bank,Axis Bank ATM,"Axis Bank, Sohna Rd",BM College of Technology,BM College of Technology & Mgmt,BML Munjal University (BMU),BOB ATM,BSF Golf Course,Baba Kanala Chowk,Badsa AMS Hospital,Badshahpur Sohna Hwy,Badshahpur Sohna Rd Hwy,"Badshahpur Sohna Rd Hwy, Haryana","Badshahpur Sohna Rd Hwy, Malibu Town","Badshahpur Sohna Rd Hwy, Raghav Vatika","Badshahpur Sohna Rd Hwy, Rajoria Ngr","Badshahpur Sohna Rd Hwy, Sector 68","Badshahpur Sohna Rd Hwy,Sector 48",Badshapur Sohna Highway,Baghera University,Bajghera Road,Bal Bharati Public School,Balaji Hospital,Bamroli Cricket Ground,Banjara Market Gurugram,Bank Of Baroda,Bank Of Baroda ATM,Basai Dhancourt Railway Station,Basai Dhankot,Basai Dhankot Railway Station,Basai Enclave Park,Basai Metro Station,Basai Road,Basant Valley Global School,Best IVF Centre,Bestech Business Tower,Bestech Central Square Mall,Bharat Petrol Pump,Bharat Petroleum Petrol Pump,Bharat Petroleum Retail Outlet,Bharat Petroleum Shree Shyam Filling,Bharat Ram Global School,Bharat Singh fuel company,Bharti International Convent School,Bhondsi Nature Park,BigBazaar,Bijwasan Railway Station,Biryani Shah,Blue Bells Public School,Broadways International School,CANARA BANK,CBR Cricket Ground,CD International School,CK Birla Hospital,"CK Birla Hospital, Gurgaon",CNG Petrol Pump,Cambridge College Of Education,Cambridge Montessori,Cambridge Montessori Preschool,Cambridge Pre-School,Canara Bank,Canara Bank - Nawada Fatehpur,Canara Bank ATM,Canara Bank New Palam Vihar,"Candor TechSpace, Sector 48",Candor Techspace,Capital Business Park,Capital Cyberscape,Captain Chandan Lal Marg,Central Bank Of India Sohna Rd,Central Park Flower Valley,Central Park II Road,Central Park Resorts,"Central Park, Sohna Rd",Central Peripheral Road,Central Plaza Mall,Centrum Plaza,Chauma Road,Cherub's Cradle,Children Park,Chirag Hospital,Choice pharmacy,Citibank ATM,City Hospital,City Square,Civil Hospital,Cloudnine Hospital Sector 47,CoNexus.Life B35,Colonel's Central Academy,Conscient One,Conscient One Mall,Cool Deck Coffee,Country Inn,Country Inn & Suites By Radisson,Country Inn & Suites by Radisson,Country Inn and Suites by Radisson,Creative Tennis Academy,Cricket Academy,Cyber ​​Park,DLC Cricket Ground,DLF Corporate Greens,DLF Corporate Park,DLF Cyber City,DLF Golf and Country Club,DLF Grand Mall,DLF Linear Park,DLF Site central office,DLF World Tech Park,DLF5 Summit Plaza,DPG Degree and Engineering College,DPS,DPS International Edge,DPS International School,DPS Manesar,DPS Sector 103,DPSG Palam Vihar,DPSG Palam Vihar Gurugram,DSD College,Damdama Lake,Damdama Lake Rd,Damdama More

In [156]:
# Replace all occurrences of 0 with maximum distance 54500.0
location_df.replace(0, 54500, inplace=True)

In [157]:
location_df.head()

,AIIMS,AIIMS Jhajjar,AIPL Business Centre,AIPL Business Club,AIPL Business Club Sector 62,AIPL Business Co Working Space,AIPL Business Tower,AIPL Joy Street Mall,APJ Abdul Kalam Park,ASF Insignia SEZ,Aapno Ghar,AapnoGhar,Aarvy Healthcare,Aarvy Healthcare Hospital,Aarvy Healthcare Super Speciality,Aarvy Hospital,Aatish Hospital,Accenture DDC5,Adarsh Senior Secondary School,"Adarsh public school,Garhi Harsaru",Agri Business Management Collage,Airia Mall,Ajit Stadium Dhanwapur,Alfaa Health Care Hospital,Alpine Convent School,Alpine Hospital,Alpine School,Altrade Business Centre,Aman Hospital,Aman Hospital & Surgical Centre,Ambience Mall,Ambience Mall New,Ambience Public School,American Express,Amity,Amity University,Amity University Gurugram,Amma Hospital,Anand Multispeciality Hospital,Anand Preschool,Ananta Hospital,Ansal Plaza,Anya Gurgaon,Ap Sports cricket ground,Apex Plus Hospital,Apollo,Apollo Pharmacy,Approved Sector 37 Mero Station,Appu Ghar,Appu Ghar Water Park,Aradhya Cricket Club Gurgaon,Aravali Adventure Hill,Aravalli Hill View Point,Aravalli Hills,Arc Hospital,Arc Multi Speciality,Arc Multi Speciality Hospital,Ardee Mall,Artemis,Artemis Hospital,Artemis Hospital Gurgaon,Artimis hospital,Arvy Hospital,Aryan Hospital,Ascendas OneHub Gurgaon,Ascendas OneHub Gurgaon Business Park,Ashiana Anmol Kid Centric Homes,Ashoka International School,Athena,Au Grand Air,Axis Bank,Axis Bank ATM,"Axis Bank, Sohna Rd",BM College of Technology,BM College of Technology & Mgmt,BML Munjal University (BMU),BOB ATM,BSF Golf Course,Baba Kanala Chowk,Badsa AMS Hospital,Badshahpur Sohna Hwy,Badshahpur Sohna Rd Hwy,"Badshahpur Sohna Rd Hwy, Haryana","Badshahpur Sohna Rd Hwy, Malibu Town","Badshahpur Sohna Rd Hwy, Raghav Vatika","Badshahpur Sohna Rd Hwy, Rajoria Ngr","Badshahpur Sohna Rd Hwy, Sector 68","Badshahpur Sohna Rd Hwy,Sector 48",Badshapur Sohna Highway,Baghera University,Bajghera Road,Bal Bharati Public School,Balaji Hospital,Bamroli Cricket Ground,Banjara Market Gurugram,Bank Of Baroda,Bank Of Baroda ATM,Basai Dhancourt Railway Station,Basai Dhankot,Basai Dhankot Railway Station,Basai Enclave Park,Basai Metro Station,Basai Road,Basant Valley Global School,Best IVF Centre,Bestech Business Tower,Bestech Central Square Mall,Bharat Petrol Pump,Bharat Petroleum Petrol Pump,Bharat Petroleum Retail Outlet,Bharat Petroleum Shree Shyam Filling,Bharat Ram Global School,Bharat Singh fuel company,Bharti International Convent School,Bhondsi Nature Park,BigBazaar,Bijwasan Railway Station,Biryani Shah,Blue Bells Public School,Broadways International School,CANARA BANK,CBR Cricket Ground,CD International School,CK Birla Hospital,"CK Birla Hospital, Gurgaon",CNG Petrol Pump,Cambridge College Of Education,Cambridge Montessori,Cambridge Montessori Preschool,Cambridge Pre-School,Canara Bank,Canara Bank - Nawada Fatehpur,Canara Bank ATM,Canara Bank New Palam Vihar,"Candor TechSpace, Sector 48",Candor Techspace,Capital Business Park,Capital Cyberscape,Captain Chandan Lal Marg,Central Bank Of India Sohna Rd,Central Park Flower Valley,Central Park II Road,Central Park Resorts,"Central Park, Sohna Rd",Central Peripheral Road,Central Plaza Mall,Centrum Plaza,Chauma Road,Cherub's Cradle,Children Park,Chirag Hospital,Choice pharmacy,Citibank ATM,City Hospital,City Square,Civil Hospital,Cloudnine Hospital Sector 47,CoNexus.Life B35,Colonel's Central Academy,Conscient One,Conscient One Mall,Cool Deck Coffee,Country Inn,Country Inn & Suites By Radisson,Country Inn & Suites by Radisson,Country Inn and Suites by Radisson,Creative Tennis Academy,Cricket Academy,Cyber ​​Park,DLC Cricket Ground,DLF Corporate Greens,DLF Corporate Park,DLF Cyber City,DLF Golf and Country Club,DLF Grand Mall,DLF Linear Park,DLF Site central office,DLF World Tech Park,DLF5 Summit Plaza,DPG Degree and Engineering College,DPS,DPS International Edge,DPS International School,DPS Manesar,DPS Sector 103,DPSG Palam Vihar,DPSG Palam Vihar Gurugram,DSD College,Damdama Lake,Damdama Lake Rd,Damdama More

In [158]:
selected_location = 'AIPL Business Tower'
selected_radius = 10

# Filtering locations within the specified radius and sorting the results
results = location_df[location_df[selected_location] < selected_radius * 1000][selected_location].sort_values()
if results.empty:
    print("Sorry! There are no posted Apartments for sale in the local neighborhood")

# Iterating over the results and printing information
for key, value in results.items():
    result_val = str(key) + " Apartments in " + str(round(value / 1000)) + " Kms"
    if value < 1000:
        print("{} Apartments is within 1 km".format(key))
        break
    print(result_val)


Signature Global City 93 Apartments is within 1 km


In [159]:
results

PropertyName
Signature Global City 93    160.0
Name: AIPL Business Tower, dtype: float64

In [160]:
# Creating an empty Series
empty_series = pd.Series([], name="AIIMS", dtype=float)

# Check if the Series is empty
if empty_series.empty:
    print("No")
else:
    print(empty_series)

No


In [161]:
type(value)

float

In [162]:
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
location_df_standardized = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns, index=location_df.index)

In [163]:
location_df_standardized.head()

,AIIMS,AIIMS Jhajjar,AIPL Business Centre,AIPL Business Club,AIPL Business Club Sector 62,AIPL Business Co Working Space,AIPL Business Tower,AIPL Joy Street Mall,APJ Abdul Kalam Park,ASF Insignia SEZ,Aapno Ghar,AapnoGhar,Aarvy Healthcare,Aarvy Healthcare Hospital,Aarvy Healthcare Super Speciality,Aarvy Hospital,Aatish Hospital,Accenture DDC5,Adarsh Senior Secondary School,"Adarsh public school,Garhi Harsaru",Agri Business Management Collage,Airia Mall,Ajit Stadium Dhanwapur,Alfaa Health Care Hospital,Alpine Convent School,Alpine Hospital,Alpine School,Altrade Business Centre,Aman Hospital,Aman Hospital & Surgical Centre,Ambience Mall,Ambience Mall New,Ambience Public School,American Express,Amity,Amity University,Amity University Gurugram,Amma Hospital,Anand Multispeciality Hospital,Anand Preschool,Ananta Hospital,Ansal Plaza,Anya Gurgaon,Ap Sports cricket ground,Apex Plus Hospital,Apollo,Apollo Pharmacy,Approved Sector 37 Mero Station,Appu Ghar,Appu Ghar Water Park,Aradhya Cricket Club Gurgaon,Aravali Adventure Hill,Aravalli Hill View Point,Aravalli Hills,Arc Hospital,Arc Multi Speciality,Arc Multi Speciality Hospital,Ardee Mall,Artemis,Artemis Hospital,Artemis Hospital Gurgaon,Artimis hospital,Arvy Hospital,Aryan Hospital,Ascendas OneHub Gurgaon,Ascendas OneHub Gurgaon Business Park,Ashiana Anmol Kid Centric Homes,Ashoka International School,Athena,Au Grand Air,Axis Bank,Axis Bank ATM,"Axis Bank, Sohna Rd",BM College of Technology,BM College of Technology & Mgmt,BML Munjal University (BMU),BOB ATM,BSF Golf Course,Baba Kanala Chowk,Badsa AMS Hospital,Badshahpur Sohna Hwy,Badshahpur Sohna Rd Hwy,"Badshahpur Sohna Rd Hwy, Haryana","Badshahpur Sohna Rd Hwy, Malibu Town","Badshahpur Sohna Rd Hwy, Raghav Vatika","Badshahpur Sohna Rd Hwy, Rajoria Ngr","Badshahpur Sohna Rd Hwy, Sector 68","Badshahpur Sohna Rd Hwy,Sector 48",Badshapur Sohna Highway,Baghera University,Bajghera Road,Bal Bharati Public School,Balaji Hospital,Bamroli Cricket Ground,Banjara Market Gurugram,Bank Of Baroda,Bank Of Baroda ATM,Basai Dhancourt Railway Station,Basai Dhankot,Basai Dhankot Railway Station,Basai Enclave Park,Basai Metro Station,Basai Road,Basant Valley Global School,Best IVF Centre,Bestech Business Tower,Bestech Central Square Mall,Bharat Petrol Pump,Bharat Petroleum Petrol Pump,Bharat Petroleum Retail Outlet,Bharat Petroleum Shree Shyam Filling,Bharat Ram Global School,Bharat Singh fuel company,Bharti International Convent School,Bhondsi Nature Park,BigBazaar,Bijwasan Railway Station,Biryani Shah,Blue Bells Public School,Broadways International School,CANARA BANK,CBR Cricket Ground,CD International School,CK Birla Hospital,"CK Birla Hospital, Gurgaon",CNG Petrol Pump,Cambridge College Of Education,Cambridge Montessori,Cambridge Montessori Preschool,Cambridge Pre-School,Canara Bank,Canara Bank - Nawada Fatehpur,Canara Bank ATM,Canara Bank New Palam Vihar,"Candor TechSpace, Sector 48",Candor Techspace,Capital Business Park,Capital Cyberscape,Captain Chandan Lal Marg,Central Bank Of India Sohna Rd,Central Park Flower Valley,Central Park II Road,Central Park Resorts,"Central Park, Sohna Rd",Central Peripheral Road,Central Plaza Mall,Centrum Plaza,Chauma Road,Cherub's Cradle,Children Park,Chirag Hospital,Choice pharmacy,Citibank ATM,City Hospital,City Square,Civil Hospital,Cloudnine Hospital Sector 47,CoNexus.Life B35,Colonel's Central Academy,Conscient One,Conscient One Mall,Cool Deck Coffee,Country Inn,Country Inn & Suites By Radisson,Country Inn & Suites by Radisson,Country Inn and Suites by Radisson,Creative Tennis Academy,Cricket Academy,Cyber ​​Park,DLC Cricket Ground,DLF Corporate Greens,DLF Corporate Park,DLF Cyber City,DLF Golf and Country Club,DLF Grand Mall,DLF Linear Park,DLF Site central office,DLF World Tech Park,DLF5 Summit Plaza,DPG Degree and Engineering College,DPS,DPS International Edge,DPS International School,DPS Manesar,DPS Sector 103,DPSG Palam Vihar,DPSG Palam Vihar Gurugram,DSD College,Damdama Lake,Damdama Lake Rd,Damdama More

In [164]:
cosine_Location = cosine_similarity(location_df_standardized)

In [165]:
cosine_Location.shape

(246, 246)

In [166]:
(3*cosine_Location + 5*cosine_PriceDetails + 6*cosine_TopFacilities).shape

(246, 246)

In [174]:
def recommend_properties_with_scores(property_name, top_n=15):

    cosine_sim_matrix = 3*cosine_TopFacilities + 2*cosine_PriceDetails + 2*cosine_Location
    # cosine_sim_matrix = cosine_Location

    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_standardized.index.get_loc(property_name)]))

    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]

    # Retrieve the names of the top properties using the indices
    top_properties = location_df_standardized.index[top_indices].tolist()

    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })

    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('Ireo Victory Valley')

,PropertyName,SimilarityScore
0,Pioneer Urban Presidia,2.810302
1,Ambience Creacions,2.778653
2,DLF The Crest,2.409491
3,Pioneer Araya,2.333655
4,Bestech Park View Grand Spa,2.211831
5,SS The Leaf,2.105677
6,Silverglades The Melia,2.084902
7,AIPL The Peaceful Homes,2.072445
8,Experion Windchants,1.982075
9,Indiabulls Centrum Park,1.967550


In [168]:
# import pickle

# pickle.dump(location_df, open('location_df.pkl','wb'))

In [172]:
# DF
import pickle

pickle.dump(location_df_standardized, open('location_df_standardized.pkl','wb'))

In [170]:
#
import pickle

pickle.dump(cosine_TopFacilities, open('Cosine_similarity_TopFacilities.pkl','wb'))
pickle.dump(cosine_PriceDetails, open('Cosine_similarity_PriceDetails.pkl','wb'))
pickle.dump(cosine_Location, open('Cosine_similarity_Location.pkl','wb'))
